In [1]:
pip install jsonlines

Note: you may need to restart the kernel to use updated packages.


In [3]:
    import json
    import os
    from typing import List, Dict, Tuple
    import random
    from transformers import AutoTokenizer, AutoModelForCausalLM
    import torch
    from tqdm import tqdm
    import re
    from collections import defaultdict, Counter
    import time
    import logging
    import jsonlines
        
    # 检查环境和 GPU 信息
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"Total GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    
    # 设置设备
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # 1. 加载 GLM-4-9B 模型和分词器
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"  # 确认 GLM-4-9B 模型名称
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.padding_side = 'left'  # 确保与 Qwen2 一致
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,  # 使用 float16 减少显存
        device_map="cuda:0",  # 强制加载到 P100
        trust_remote_code=True
    )
    model.eval()
    print("Model loaded successfully")
    print(f"Model device: {next(model.parameters()).device}")

PyTorch version: 2.5.1+cu124
CUDA available: True
CUDA version: 12.4
GPU: Tesla P100-PCIE-16GB
Total GPU memory: 15.89 GB
Using device: cuda:0


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

2025-05-12 11:15:37.428985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747048537.615771      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747048537.665063      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Model loaded successfully
Model device: cuda:0


In [4]:
# 打印初始显存
if torch.cuda.is_available():
    print(f"GPU memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"GPU memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")

# 2. 构造提示函数
def create_zero_shot_prompt(passage: str, number: str) -> str:
    return f"""Answer with only 'Yes' or 'No'. Do not provide explanations. Is "{number}" in the following passage an error? "{passage}"
Answer:"""

def create_few_shot_prompt(passage: str, number: str) -> str:
    examples = [
        {"passage": "Spiders have 9 limbs.", "number": "9", "answer": "Yes"},
        {"passage": "Spiders have 8 limbs.", "number": "8", "answer": "No"},
        {"passage": "Mike's height is -3.6 meters.", "number": "-3.6", "answer": "Yes"},
        {"passage": "Mike's height is 1.8 meters.", "number": "1.8", "answer": "No"}
    ]
    prompt = "Answer with only 'Yes' or 'No'. Do not provide explanations.\n"
    for ex in examples:
        prompt += f"""Question: Is "{ex['number']}" in the following passage an error? "{ex['passage']}"
Answer: {ex['answer']}\n"""
    prompt += f"""Question: Is "{number}" in the following passage an error? "{passage}"
Answer:"""
    return prompt

# 3. 加载 BeNEDect 数据集
def load_benedect_dataset(file_path: str) -> List[Dict]:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"数据集文件 {file_path} 不存在，请确认路径！")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            dataset_dict = json.load(f)
    except json.JSONDecodeError as e:
        raise ValueError(f"JSON 文件解析错误：{e}")
    
    dataset = list(dataset_dict.values())
    save_list = []
    
    for i, data in enumerate(tqdm(dataset, desc="Processing dataset")):
        required_fields = ['correct_number', 'correct_passage', 'error_number', 'error_passage', 'dataset', 'operation']
        for field in required_fields:
            if field not in data:
                print(f"样本 {data.get('id', '未知')} 缺少字段 {field}")
                continue
        
        prompt_fn = create_few_shot_prompt if i % 48 == 0 else create_zero_shot_prompt
        correct_item = {
            "prompt": prompt_fn(data['correct_passage'], data['correct_number']),
            "expected_answer": "No",
            "dataset": data['dataset'],
            "operation": data['operation'],
            "error_annotation": data.get('error_annotation', {}),
            "passage": data['correct_passage'],
            "number": data['correct_number'],
            "prompt_type": "few_shot" if i % 48 == 0 else "zero_shot"
        }
        error_item = {
            "prompt": prompt_fn(data['error_passage'], data['error_number']),
            "expected_answer": "Yes",
            "dataset": data['dataset'],
            "operation": data['operation'],
            "error_annotation": data.get('error_annotation', {}),
            "passage": data['error_passage'],
            "number": data['error_number'],
            "prompt_type": "few_shot" if i % 48 == 0 else "zero_shot"
        }
        save_list.append(correct_item)
        save_list.append(error_item)
    
    return save_list

# 4. 单条推理
def predict_single(prompt: str, max_retries: int = 3) -> str:
    print(f"Single prediction prompt: {prompt[:100]}...")
    attempt = 0
    success = False
    prediction = None
    
    while attempt < max_retries and not success:
        try:
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            print(f"Input device: {inputs['input_ids'].device}")
            print(f"Input shape: {inputs['input_ids'].shape}")
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=5,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                    do_sample=False,
                    top_k=1,
                    top_p=0.0
                )
            
            prediction = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip()
            success = True
            print(f"Single prediction success: Raw Prediction: {prediction}")
            
        except RuntimeError as e:
            print(f"单条推理失败（尝试 {attempt + 1}/{max_retries}）：{e}")
            attempt += 1
            torch.cuda.empty_cache()
            time.sleep(1)
            if attempt == max_retries:
                print("单条推理失败，跳过")
                prediction = "generation_error"
        
        finally:
            if 'inputs' in locals():
                for v in inputs.values():
                    del v
            torch.cuda.empty_cache()
            print(f"GPU memory after single prediction: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    
    return prediction

# 5. 批次推理
def predict_batch(prompts: List[str], batch_size: int = 8, max_retries: int = 3) -> List[str]:
    predictions = []
    
    for i in tqdm(range(0, len(prompts), batch_size), desc="Predicting"):
        batch_prompts = prompts[i:i + batch_size]
        attempt = 0
        success = False
        batch_preds = None
        
        while attempt < max_retries and not success:
            try:
                # 强制统一序列长度，检查张量形状
                inputs = tokenizer(
                    batch_prompts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=512,
                    return_attention_mask=True
                )
                inputs = {k: v.to(device) for k, v in inputs.items()}
                print(f"Batch {i//batch_size} input shapes: input_ids={inputs['input_ids'].shape}, attention_mask={inputs['attention_mask'].shape}")
                
                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=5,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        do_sample=False,
                        top_k=1,
                        top_p=0.0
                    )
                
                batch_preds = [
                    tokenizer.decode(output[inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip()
                    for output in outputs
                ]
                success = True
                if i % (10 * batch_size) == 0:
                    print(f"批次 {i//batch_size}: 原始预测: {batch_preds}")
                
            except RuntimeError as e:
                print(f"批次推理失败（尝试 {attempt + 1}/{max_retries}）：{e}")
                attempt += 1
                torch.cuda.empty_cache()
                time.sleep(1)
                if attempt == max_retries:
                    print(f"批次 {i//batch_size} 推理失败，跳过")
                    batch_preds = ["generation_error"] * len(batch_prompts)
            
            finally:
                if 'inputs' in locals():
                    for v in inputs.values():
                        del v
                torch.cuda.empty_cache()
                print(f"GPU memory after batch {i//batch_size}: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
        
        predictions.extend(batch_preds)
    
    return predictions

# 6. 保存预测结果到 JSONL
def save_predictions_to_jsonl(data: List[Dict], predictions: List[str], output_file: str):
    with jsonlines.open(output_file, mode='w') as writer:
        for item, pred in zip(data, predictions):
            result = {
                "prompt": item['prompt'],
                "passage": item['passage'],
                "number": item['number'],
                "expected_answer": item['expected_answer'],
                "raw_prediction": pred,
                "dataset": item['dataset'],
                "operation": item['operation'],
                "error_annotation": item['error_annotation'],
                "prompt_type": item['prompt_type']
            }
            writer.write(result)

GPU memory allocated: 14.96 GB
GPU memory reserved: 15.08 GB


In [6]:
file_path = "/kaggle/input/benedect/BeNEDect_all.json"  # 确认路径
dataset = load_benedect_dataset(file_path)

random_sample = random.choice(dataset)
print("=== 随机样本推理 ===")
print(f"Passage: {random_sample['passage']}")
print(f"Number: {random_sample['number']}")
print(f"Expected Answer: {random_sample['expected_answer']}")
print(f"Prompt Type: {random_sample['prompt_type']}")
print(f"Prompt:\n{random_sample['prompt']}")

random_pred = predict_single(random_sample['prompt'])
print(f"Raw Prediction: {random_pred}")

prompts = [item['prompt'] for item in dataset]
predictions = predict_batch(prompts, batch_size=4, max_retries=3)  # 减小批次大小
output_file = "/kaggle/working/predictions.jsonl"
save_predictions_to_jsonl(dataset, predictions, output_file)
print(f"预测结果已保存到 {output_file}")

Processing dataset: 100%|██████████| 4800/4800 [00:00<00:00, 223857.89it/s]


=== 随机样本推理 ===
Passage: Renaissance sculpture proper is often taken to begin with the famous competition for the doors of the Florence Baptistry in 1403, from which the trial models submitted by the winner, Lorenzo Ghiberti, and Filippo Brunelleschi survive. Ghiberti's doors are still in place, but were undoubtedly eclipsed by his second pair for the other entrance, the so-called "Gates of Paradise", which took him from 1425 to 1452, and are dazzlingly confident classicizing compositions with varied depths of relief allowing extensive backgrounds. The intervening years had seen Ghiberti's early assistant Donatello develop with seminal statues including his "Davids" in marble (1408–09) and bronze (129.60s), and his Equestrian statue of Gattamelata, as well as reliefs. A leading figure in the later period was Andrea del Verrocchio, best known for his equestrian statue of Bartolomeo Colleoni in Venice; his pupil Leonardo da Vinci designed an equine sculpture in 1482 "The Horse" for Milan-

Predicting:   0%|          | 0/2400 [00:00<?, ?it/s]

Batch 0 input shapes: input_ids=torch.Size([4, 164]), attention_mask=torch.Size([4, 164])


Predicting:   0%|          | 1/2400 [00:02<1:25:42,  2.14s/it]

...

Predicting: 100%|█████████▉| 2399/2400 [1:24:42<00:01,  1.23s/it]

GPU memory after batch 2398: 14.97 GB
Batch 2399 input shapes: input_ids=torch.Size([4, 66]), attention_mask=torch.Size([4, 66])


Predicting: 100%|██████████| 2400/2400 [1:24:43<00:00,  2.12s/it]

GPU memory after batch 2399: 14.97 GB


预测结果已保存到 /kaggle/working/predictions.jsonl


In [1]:
import json
from collections import Counter, defaultdict
from typing import List, Dict, Tuple

In [2]:
def parse_prediction(raw_prediction: str) -> str:
    raw_prediction = raw_prediction.lower()
    if 'yes' in raw_prediction:
        return 'yes'
    elif 'no' in raw_prediction:
        return 'no'
    else:
        # print(f"无法解析响应: {raw_prediction}")
        return 'generation_error'

def evaluate_model(data_list: List[Dict], unparsed_output_file: str = 'unparsed_predictions.json') -> Tuple[Dict, Dict]:
    metrics = Counter()
    detailed_metrics = {
        'by_domain': defaultdict(Counter),
        'by_error_type': defaultdict(Counter),
        'by_operation': defaultdict(Counter),
        'by_prompt_type': defaultdict(Counter)
    }
    unparsed_data = {}  # 存储无法解析的样本，格式为 {id: {...}}
    
    for idx, item in enumerate(data_list):
        expected = item['expected_answer'].lower()  # Yes/No 转为小写
        pred = parse_prediction(item['raw_prediction'])
        item['parsel_prediction'] = pred  # 保存解析结果
        
        # 如果无法解析，添加到 unparsed_data
        if pred == 'generation_error':
            # 只保存 expected_answer == "Yes" 的样本（错误样本）
            if expected == 'yes':
                sample_id = f"unparsed_{idx}"
                unparsed_data[sample_id] = {
                    "error_number": item['number'],
                    "error_passage": item['passage'],
                    "dataset": item['dataset'],
                    "operation": item['operation'],
                    "error_annotation": item['error_annotation'],
                    # 以下字段需补充（若有正确数据）
                    "correct_number": "",  # 需手动补充或从原始数据推导
                    "correct_passage": ""  # 需手动补充或从原始数据推导
                }
        
        domain = item['dataset']
        operation = item['operation']
        prompt_type = item['prompt_type']
        error_types = [k for k, v in item['error_annotation'].items() if v > 0]
        
        # 计算总体指标
        if pred == expected:
            if expected == 'yes':
                metrics['TP'] += 1
                for et in error_types:
                    detailed_metrics['by_error_type'][et]['TP'] += 1
                detailed_metrics['by_domain'][domain]['TP'] += 1
                detailed_metrics['by_operation'][operation]['TP'] += 1
                detailed_metrics['by_prompt_type'][prompt_type]['TP'] += 1
            else:  # expected == 'no'
                metrics['TN'] += 1
                for et in error_types:
                    detailed_metrics['by_error_type'][et]['TN'] += 1
                detailed_metrics['by_domain'][domain]['TN'] += 1
                detailed_metrics['by_operation'][operation]['TN'] += 1
                detailed_metrics['by_prompt_type'][prompt_type]['TN'] += 1
        else:
            if expected == 'yes':
                metrics['FN'] += 1
                for et in error_types:
                    detailed_metrics['by_error_type'][et]['FN'] += 1
                detailed_metrics['by_domain'][domain]['FN'] += 1
                detailed_metrics['by_operation'][operation]['FN'] += 1
                detailed_metrics['by_prompt_type'][prompt_type]['FN'] += 1
            else:  # expected == 'no'
                metrics['FP'] += 1
                for et in error_types:
                    detailed_metrics['by_error_type'][et]['FP'] += 1
                detailed_metrics['by_domain'][domain]['FP'] += 1
                detailed_metrics['by_operation'][operation]['FP'] += 1
                detailed_metrics['by_prompt_type'][prompt_type]['FP'] += 1
        
        if pred == 'generation_error':
            metrics['Generation Error'] += 1
            for et in error_types:
                detailed_metrics['by_error_type'][et]['Generation Error'] += 1
            detailed_metrics['by_domain'][domain]['Generation Error'] += 1
            detailed_metrics['by_operation'][operation]['Generation Error'] += 1
            detailed_metrics['by_prompt_type'][prompt_type]['Generation Error'] += 1
    
    # 保存无法解析的数据到 JSON
    if unparsed_data:
        with open(unparsed_output_file, 'w', encoding='utf-8') as f:
            json.dump(unparsed_data, f, indent=2, ensure_ascii=False)
        # print(f"无法解析的 {len(unparsed_data)} 条数据已保存到 {unparsed_output_file}")
        print("注意：JSON 文件仅包含 expected_answer='Yes' 的样本，correct_number 和 correct_passage 需手动补充")
    else:
        print("没有无法解析的数据")
    
    total = len(data_list)
    metrics['Accuracy'] = (metrics['TP'] + metrics['TN']) / total if total > 0 else 0
    return metrics, detailed_metrics

In [3]:
# 读取 predictions.jsonl
data_list = []
input_file = 'deepseek_predictions.jsonl'  # 确认路径
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        data_list.append(data)

# 评测模型并保存无法解析的数据
unparsed_output_file = 'deepseek_unparsed_predictions.json'
metrics, detailed_metrics = evaluate_model(data_list, unparsed_output_file)

# 打印总体指标
print("\nOverall Metrics:")
total = len(data_list)
for key, value in metrics.items():
    if key == 'Accuracy':
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value} ({value / total:.3f})")

# 打印分维度指标
print("\nMetrics by Domain:")
for domain, counts in detailed_metrics['by_domain'].items():
    print(f"{domain}: {dict(counts)}")

print("\nMetrics by Error Type:")
for error_type, counts in detailed_metrics['by_error_type'].items():
    print(f"{error_type}: {dict(counts)}")

print("\nMetrics by Operation:")
for operation, counts in detailed_metrics['by_operation'].items():
    print(f"{operation}: {dict(counts)}")

print("\nMetrics by Prompt Type:")
for prompt_type, counts in detailed_metrics['by_prompt_type'].items():
    print(f"{prompt_type}: {dict(counts)}")

注意：JSON 文件仅包含 expected_answer='Yes' 的样本，correct_number 和 correct_passage 需手动补充

Overall Metrics:
FP: 4207 (0.438)
TP: 3849 (0.401)
TN: 593 (0.062)
FN: 951 (0.099)
Generation Error: 509 (0.053)
Accuracy: 0.463

Metrics by Domain:
Numeracy_600K_article_title: {'FP': 860, 'TP': 822, 'TN': 140, 'FN': 178}
aclsent: {'FP': 905, 'TP': 905, 'FN': 43, 'TN': 43}
DROP: {'TN': 180, 'FN': 474, 'FP': 814, 'TP': 520, 'Generation Error': 499}
qa-text-source-comparison: {'TN': 67, 'FN': 92, 'FP': 857, 'TP': 832, 'Generation Error': 8}
FinNum: {'FP': 771, 'TP': 770, 'TN': 163, 'FN': 164, 'Generation Error': 2}

Metrics by Error Type:
Error in Number Relationships: {'FP': 172, 'TP': 161, 'TN': 24, 'FN': 35, 'Generation Error': 4}
Undetectable Error: {'FP': 395, 'TP': 387, 'TN': 69, 'FN': 77, 'Generation Error': 14}
Type Error: {'FP': 436, 'TP': 415, 'TN': 82, 'FN': 103, 'Generation Error': 10}
Anomaly: {'FP': 203, 'TP': 186, 'TN': 27, 'FN': 44, 'Generation Error': 10}
Improper Data: {'FP': 26, 'TP': 27, 